# Topological Data Analysis
Matias Ollu, Yuguang Yao

## Abstact
This resipotary aims to present our result to solve the problem of the project of topological data analysis in CSC_42021_EP - Conception et analyse d'algorithmes (2024-2025) of the École Polytechnique. We present an alogritme to compute the simplexes and the filtration value of a given $\alpha$-complex or Čech-complex of a arbitary point-set. We devolepped an alogritme general enough to calculate the circumcircle of $n$ points in $d$-dimension euclidean space.

## Environment
We utilise the packets following:

In [ ]:
import random
import numpy as np
from itertools import combinations

## $\alpha$-complex and Čech-complex
We will firstly define the key concepts that we are going to use.

* Define 1: 

## Question 1

>Task 1. Implement the computation of the minimal enclosing ball (MEB)
>of a set of points using one (or more) of the algorithms from https:// en.
>wikipedia.org/ wiki/ LP-type_problem.

We defined a class of Sphere(center,radius), and uitilised the alogithm of Weltz(LP-Problem) to solve the MEB quetion and we solved alse the circumcircle of $n$ points in $d$-dimension euclidean space.

In [ ]:
# This class is used to represent a sphere.
class Sphere:
    def __init__(self, center, radius):
        self.center = np.array(center)
        self.radius = radius

    def contains(self, point):
        """Check if the sphere contains the given point."""
        return np.linalg.norm(self.center - np.array(point)) <= self.radius #+ 1e-9
    
    def contains_strict(self,point):
        """Check if the sphere contains the given point."""
        norme = np.linalg.norm(self.center - np.array(point))
        return not (np.isclose((norme),self.radius) or norme>self.radius+1e-9)
    
    def onradius(self,point):
        """Check if the point is on the sphere"""
        return np.isclose(np.linalg.norm(self.center - np.array(point)),self.radius)

On déduit les équation pour calculer des circumcircles:

Soit $P_i^n$ le $i$-ième coordinée de point $P^n$ dans un emsemble des $N$-points dans $d$-dimension en satisfasant $1 \leq n \leq N \leq d+1$. Et soit $c=\sum_n k_n P^{n0}$. Pour la centre du circumcircle, pour quelconque $n,m\in[1,n]\cap Z$  ,on a:

$$|c-P^n|=|c-P^m| $$
$$\sum_i (c_i-P_i^n)^2-(n\to m)=0 $$
$$\sum_i (P_i^n)^2+2P_i^nc_i-(n\to m)=0 $$
$$\sum_i ((P_i^n)^2-(P_i^m)^2)=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  $$
$$|P^n|-|P^m|=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  .$$

Dans l'implementation, on prends $m=0$
$$|P^n|-|P^0|=\sum_l (2P_i^l\sum_i (P_i^n-P_i^0)) k_l  $$

$$A_{mn}=2P^{m0} \cdot P^{n0} $$
$$b_n=|P^n|-|P^0|.$$

c'est une systeme linaire avec $n-1$ équations et $n-1$ variables(theorem à prouver). Apres qu'on resoudre la systeme, on peut extrait  $c=\sum_n k_n P^{n0}$ par $c=k \cdot P+P^0$.

In [ ]:
def make_sphere_n_points(points):
    """
    Trouver le minimal circumcircle dans l'espace d-dimension de n points (n <= d+1).
    
    when utilising this mathod, we assume that the points are not colinear and that the dimension is at least 2.

    Paramètres :
        points: np.ndarray, shape (n, d)
    Retourne :
        Sphere
    """
    points = np.array(points, dtype=float)
    
    # Calcul de la matrice A et du vecteur b
    diffs = points[1:] - points[0]  # Différences (P^n - P^0) pour n = 1, ..., N-1
    A = 2 * np.dot(diffs, diffs.T)  # Matrice A
    b = np.sum(diffs ** 2, axis=1)  # Vecteur b

    # Résolution du système linéaire pour trouver les coefficients k
    k = np.linalg.solve(A, b)

    # Calculer le centre
    center = points[0] + np.dot(k, diffs)

    # Calculer le rayon
    radius = np.linalg.norm(center - points[0])

    # Retourner une instance de Sphere
    return Sphere(center=center, radius=radius)

In [ ]:
def trivial(R):
    """Find the minimal sphere for 0, 1 or mores points."""
    if not R:
        return Sphere([0, 0, 0], 0)
    elif len(R) == 1:
        return Sphere(R[0], 0)
    elif len(R) >= 1:
        return make_sphere_n_points(R)

def welzl(P, R):
    """Recursive implementation of Welzl's algorithm for 3D."""
    if not P or len(R) == 4:
        return trivial(R)

    p = P.pop(random.randint(0, len(P) - 1))
    D = welzl(P, R)

    if D.contains(p):
        P.append(p)
        return D

    result = welzl(P, R + [p])
    P.append(p)
    return result

def minimal_enclosing_sphere(points):
    """Compute the minimal enclosing sphere for a set of points."""
    points = points[:]
    random.shuffle(points)
    return welzl(points, [])